# Import Libraries


In [1]:
import importlib
import sys
sys.modules['imp'] = importlib

In [2]:
%load_ext autoreload
# %reload_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
from IPython import display

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import roc_curve, auc, accuracy_score, classification_report, confusion_matrix, roc_auc_score
# from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
params_cfg = {
    "action"   : "train_feat01",
    "feat_path": "/content/drive/MyDrive/house_predict/process/exps/data.npz",
    "seed"    : 42, # Set random seed
    "exp_dir" : os.path.abspath('/content/drive/MyDrive/house_predict/process/exps'),
    'exp_name': '/content/drive/MyDrive/house_predict/process/exps/trainbase_24102025',
    "data_dir": os.path.abspath("/content/drive/MyDrive/house_predict/data"),
    "verbose" : True,
}

params_cfg.update(**{
    "save_dir": os.path.abspath(f'{params_cfg["exp_dir"]}/{params_cfg["exp_name"]}')
})

for v in params_cfg:
    print(f'+ {v}: {params_cfg[v]}')

globals().update(**params_cfg)

+ action: train_feat01
+ feat_path: /content/drive/MyDrive/house_predict/process/exps/data.npz
+ seed: 42
+ exp_dir: /content/drive/MyDrive/house_predict/process/exps
+ exp_name: /content/drive/MyDrive/house_predict/process/exps/trainbase_24102025
+ data_dir: /content/drive/MyDrive/house_predict/data
+ verbose: True
+ save_dir: /content/drive/MyDrive/house_predict/process/exps/content/drive/MyDrive/house_predict/process/exps/trainbase_24102025


# Data Load


In [4]:
df_train = pd.read_csv(f'{data_dir}/train.csv')
df_test = pd.read_csv(f'{data_dir}/test.csv')

if params_cfg["verbose"]:
    print("-"*10, "information", "-"*10)
    print(f'train-col: {set(df_train.columns)}')
    print(f'test-col: {set(df_test.columns)}')
    print("Union:", set(df_train.columns).intersection(set(df_test.columns)))
    print("Difference:", set(df_train.columns).difference(set(df_test.columns)))

---------- information ----------
train-col: {'BsmtUnfSF', 'FullBath', 'GarageFinish', 'BsmtFullBath', 'GarageQual', 'OverallCond', 'Id', 'MSSubClass', 'PavedDrive', 'KitchenAbvGr', 'PoolArea', 'Foundation', 'Utilities', 'SaleType', '1stFlrSF', 'EnclosedPorch', 'Street', 'GarageArea', 'BsmtFinSF1', 'LowQualFinSF', 'YrSold', 'Exterior1st', 'BsmtHalfBath', 'OpenPorchSF', 'MasVnrArea', 'GarageCars', 'BedroomAbvGr', 'Fireplaces', 'LotFrontage', 'BsmtFinType1', 'Functional', 'BsmtCond', 'YearBuilt', 'MasVnrType', 'RoofStyle', 'ScreenPorch', 'Condition2', 'GarageYrBlt', 'Fence', 'LandSlope', 'MiscFeature', 'BsmtFinType2', 'OverallQual', 'BsmtExposure', 'BldgType', 'Alley', 'GrLivArea', '2ndFlrSF', 'TotRmsAbvGrd', 'LandContour', 'LotArea', 'TotalBsmtSF', 'Electrical', 'KitchenQual', 'BsmtQual', 'FireplaceQu', 'Neighborhood', 'LotShape', 'SalePrice', 'CentralAir', 'MSZoning', 'Exterior2nd', 'Condition1', '3SsnPorch', 'Heating', 'RoofMatl', 'GarageCond', 'MoSold', 'MiscVal', 'HouseStyle', 'Year

# Processing

## Xử lí các dữ liệu thiếu trong việc phân tích EDA

### PoolQc

In [5]:
np.unique(df_train['PoolQC'].astype(str))

array(['Ex', 'Fa', 'Gd', 'nan'], dtype=object)

In [6]:
df_train['PoolQC'].fillna('NoPool', inplace=True)

**Nhận Xét:**
+ PoolQc là thuộc tính dạng string
+ Điền NoPool cho các cột NaN vì không phải nhà nào cũng có hồ bơi để đánh giá.
+ Ta sẽ mã hóa cột PoolQc dưới dạng số: "NoPool": 0, "Ex": 1, "Fa": 2, "Gd": 3.

In [7]:
pool_mapping = {'NoPool': 0, 'Ex': 1, 'Fa': 2, 'Gd': 3}

In [8]:
df_output = pd.DataFrame()
df_output['PoolQC'] = df_train['PoolQC'].apply(lambda x: pool_mapping[x])

### MiscFeature

In [9]:
np.unique(df_train['MiscFeature'].astype(str))

array(['Gar2', 'Othr', 'Shed', 'TenC', 'nan'], dtype=object)